## Setup

In [5]:
! pip install openai python-dotenv elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.6/524.6 kB 5.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [15]:
from openai import OpenAI
from elasticsearch import Elasticsearch
import json
from tqdm.auto import tqdm 

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
client = OpenAI(
    base_url = 'http://localhost:11434/v1/',#
    api_key = 'ollama'
)

## Simple RAG

In [13]:
q = 'The course has already started, can I still enroll?'

In [34]:
prompt_template = """
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ databse. 
Use only the facts in the CONTEXT when answering the question.
If the context doesn't contain the answer, output NONE.

QUESTION: 
{question}

CONTEXT: 
{context}
""".strip()

In [35]:
context = ""

for doc in results:
    context += f"section: {doc['section']} \nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [31]:
print(context)

section: General course-related questions 
question: The course has already started. Can I still join it?
answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section: General course-related questions 
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions 
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so yo

In [38]:
prompt = prompt_template.format(question=q, context = context).strip()

## Modularization

In [42]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    results = index.search(
        query = query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 5
    )   
    return results



In [45]:
#search("how do I run kafka?")

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
    You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ databse. 
    Use only the facts in the CONTEXT when answering the question.

    QUESTION: 
    {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""

    for doc in search_results:
        context += f"section: {doc['section']} \nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context = context).strip()

    return prompt

In [58]:
def llm(prompt):
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [59]:
query = "how do I run kafka?"
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

In [61]:
print(answer)

To run Kafka, you can follow these different methods based on the language you are using:
1. For Java programs, in the project directory, run:
`java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java`
2. For Python programs, such as producer.py, you can create a virtual environment and run requirements.txt and the python files in that environment by following these steps:
- To create a virtual env and install packages (run only once):
```
python -m venv env
source env/bin/activate
pip install -r ../requirements.txt
```
- To activate the virtual environment (you'll need to run it every time you need the virtual env):
```
source env/bin/activate
```
- To deactivate the virtual environment:
```
deactivate
```
3. If you encounter a permission denied error like "./build.sh: Permission denied", you can resolve it by running this command in the terminal in the same directory (/docker/spark):
`chmod +x build.sh`
4. If you face the error "ModuleNotFoundEr

## Using elastic search

Command to run elastic search  
docker run -it --name elasticsearch -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" -e "xpack.security.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.4.3

http get request to local host  
curl http://localhost:9200

In [12]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)
    
documents = []

#docs_raw[2] # 3 courses
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [ ]:
es_client = Elasticsearch('http://localhost:9200')
es_client.info()


In [13]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

In [14]:
index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [16]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:29<00:00, 31.63it/s]


In [17]:
query = 'I just discovered the course, can I still join it?'

In [18]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs


In [19]:
results = elastic_search(query)

In [20]:
def build_prompt(query, search_results):
    prompt_template = """
    You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ databse. 
    Use only the facts in the CONTEXT when answering the question.

    QUESTION: 
    {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""

    for doc in search_results:
        context += f"section: {doc['section']} \nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context = context).strip()

    return prompt
def llm(prompt):
    response = client.chat.completions.create(
        model = 'phi3',
        messages = [{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [21]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [23]:
response = rag(query)

InternalServerError: Error code: 500 - {'error': {'message': 'model requires more system memory (5.6 GiB) than is available (2.9 GiB)', 'type': 'api_error', 'param': None, 'code': None}}

In [24]:
response

NameError: name 'response' is not defined